# Vehicule ID detection

In [1]:
import geopandas as gpd
import folium
import json
from tqdm import tqdm
import re

from custom_functions.gtfs_methods import get_largest_group,get_schedule, extract_schedule
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

In [3]:
headway_qos = pd.read_pickle("data/computed/theoretical_schedule_qos.pkl")
routes = pd.read_csv('data/timetables/gtfs3Sept/routes.txt', sep = ",")
stops = pd.read_csv('data/timetables/gtfs3Sept/stops.txt', sep = ",")
calendar = pd.read_pickle("data/computed/gtfs3_/calendar.pkl")

In [4]:
headway_qos

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity
0,112360063233813600,05:17:00,05:17:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,19020,NaN,cluster_0,30.0,0
1,112360064233813600,05:50:00,05:50:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862004,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,21000,33.0,cluster_0,30.0,0
2,112360065233813600,06:19:00,06:19:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,22740,29.0,cluster_0,30.0,0
3,112360066233813600,06:49:00,06:49:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862004,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,24540,30.0,cluster_0,30.0,0
4,112360067233813600,07:19:00,07:19:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,26340,30.0,cluster_0,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819466,113543415238463502,20:43:00,20:43:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963879,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,74580,148.0,cluster_0,53.0,0
2819467,113543406238463502,21:36:00,21:36:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963876,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,77760,53.0,cluster_0,53.0,0
2819468,113543441238463502,23:35:00,23:35:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963912,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,84900,119.0,cluster_0,53.0,0
2819469,113543485238463502,23:55:00,23:55:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963877,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,86100,20.0,cluster_0,53.0,0


In [3]:

geom = gpd.GeoDataFrame(pd.read_pickle("data/computed/stops_geom.pkl"))
lines = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops_geom = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")

2022-12-06 14:23:05,466 - fiona.ogrext - INFO - collection - : Line (215) - Failed to auto identify EPSG: 7


In [5]:
calendar.label.unique()

array(['workdays', 'saturday', 'sunday'], dtype=object)

In [6]:
lines = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops_geom = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")

2022-12-05 18:14:51,966 - fiona.ogrext - INFO - collection - : Line (215) - Failed to auto identify EPSG: 7


## Creating the geometry file

In [16]:
success_stops = stops_geom.groupby(by = ["Code_Ligne", "Variante"])

grp = []

for name, group in success_stops:
    group['geom_position_before'] = group.geometry.shift(1)
    group["dist_m"] = group['geometry'].distance(group["geom_position_before"])

    temp = group.merge(
        lines,
        "left",
        left_on = ["Code_Ligne", "Variante"],
        right_on = ["LIGNE", "VARIANTE"]
    ).rename(columns={"geometry_y" : "ligne_Shape", "geometry_x" : "stop_Shape" })

    grp.append(temp)
    del temp

geom = gpd.GeoDataFrame(pd.concat(grp, ignore_index=True))
del grp


/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as BD72 / Belgian Lambert 72 (the single non-null crs provided).
  warnings.warn(
/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as BD72 / Belgian Lambert 72 (the single non-null crs provided).
  warnings.warn(
/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as BD72 / Belgian Lambert 72 (the single non-null crs provided).
  warnings.warn(
/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as BD72 / Belgian Lambert 72 (the single non-null crs provided).
  warnings.warn(
/Use

In [27]:
geom["stop_id_cleaned"] = geom["stop_id"].apply(lambda x : re.sub('\D', '', x.lstrip("0")))
geom["ligne_cleaned"] = geom["Code_Ligne"].apply(lambda x : re.sub('\D', '', x.lstrip("0")))
geom[geom.ligne_cleaned.isin(obs.route_short_name.unique())]

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape,stop_id_cleaned,ligne_cleaned
0,001m,1,1,8733,GARE DE L'OUEST,WESTSTATION,Gare de l'Ouest,Weststation,146633.5,170956.4,M,1,STOCKEL,POINT (146633.500 170956.400),None,NaN,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146...",8733,1
1,001m,1,2,8742,BEEKKANT,BEEKKANT,Beekkant,Beekkant,146776.5,171444.3,M,1,STOCKEL,POINT (146776.500 171444.300),POINT (146633.500 170956.400),508.424439,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146...",8742,1
2,001m,1,3,8292,ETANGS NOIRS,ZWARTE VIJVERS,Étangs Noirs,Zwarte Vijvers,147492.7,171859.9,M,1,STOCKEL,POINT (147492.700 171859.900),POINT (146776.500 171444.300),828.049395,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146...",8292,1
3,001m,1,4,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,Comte de Flandre,Graaf van Vlaanderen,148013.6,171590.4,M,1,STOCKEL,POINT (148013.600 171590.400),POINT (147492.700 171859.900),586.487050,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146...",8282,1
4,001m,1,5,8272,SAINTE-CATHERINE,SINT-KATELIJNE,Sainte-Catherine,Sint-Katelijne,148539.5,171278.2,M,1,STOCKEL,POINT (148539.500 171278.200),POINT (148013.600 171590.400),611.587811,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146...",8272,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591,098b,2,8,9686B,DORENT,DORENT,Dorent,Dorent,145329.5,164884.6,B,98,CERIA,POINT (145329.500 164884.600),POINT (145857.600 164769.000),540.604264,098b,2,#E87D0D,01/09/2021,06/03/2022,"LINESTRING Z (148158.700 165873.500 0.000, 148...",9686,98
3592,098b,2,9,5964,PATHE,PATHE,Pathé,Pathé,145316.0,165948.0,B,98,CERIA,POINT (145316.000 165948.000),POINT (145329.500 164884.600),1063.485689,098b,2,#E87D0D,01/09/2021,06/03/2022,"LINESTRING Z (148158.700 165873.500 0.000, 148...",5964,98
3593,098b,2,10,3815,MOZART,MOZART,Mozart,Mozart,145542.0,166700.0,B,98,CERIA,POINT (145542.000 166700.000),POINT (145316.000 165948.000),785.226082,098b,2,#E87D0D,01/09/2021,06/03/2022,"LINESTRING Z (148158.700 165873.500 0.000, 148...",3815,98
3594,098b,2,11,3851,BOLLINCKX,BOLLINCKX,Bollinckx,Bollinckx,145657.0,166998.0,B,98,CERIA,POINT (145657.000 166998.000),POINT (145542.000 166700.000),319.419786,098b,2,#E87D0D,01/09/2021,06/03/2022,"LINESTRING Z (148158.700 165873.500 0.000, 148...",3851,98


In [28]:
geom.to_pickle('data/computed/stops_geom.pkl')

## Vehicule detection

AMAZING NEWS : Theoretically speaking, each trip is actually a vehicules ... (Lucarne de hakimovich)

### Sample Model - 7 line

In [37]:
headway_qos

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity
0,112360063233813600,05:17:00,05:17:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,19020,NaN,cluster_0,30.0,0
1,112360064233813600,05:50:00,05:50:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862004,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,21000,33.0,cluster_0,30.0,0
2,112360065233813600,06:19:00,06:19:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,22740,29.0,cluster_0,30.0,0
3,112360066233813600,06:49:00,06:49:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862004,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,24540,30.0,cluster_0,30.0,0
4,112360067233813600,07:19:00,07:19:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,26340,30.0,cluster_0,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819466,113543415238463502,20:43:00,20:43:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963879,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,74580,148.0,cluster_0,53.0,0
2819467,113543406238463502,21:36:00,21:36:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963876,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,77760,53.0,cluster_0,53.0,0
2819468,113543441238463502,23:35:00,23:35:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963912,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,84900,119.0,cluster_0,53.0,0
2819469,113543485238463502,23:55:00,23:55:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963877,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,86100,20.0,cluster_0,53.0,0


In [38]:
geom

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape
0,001m,1,1,8733,GARE DE L'OUEST,WESTSTATION,Gare de l'Ouest,Weststation,146633.5,170956.4,M,1,STOCKEL,POINT (146633.500 170956.400),None,NaN,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,001m,1,2,8742,BEEKKANT,BEEKKANT,Beekkant,Beekkant,146776.5,171444.3,M,1,STOCKEL,POINT (146776.500 171444.300),POINT (146633.500 170956.400),508.424439,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
2,001m,1,3,8292,ETANGS NOIRS,ZWARTE VIJVERS,Étangs Noirs,Zwarte Vijvers,147492.7,171859.9,M,1,STOCKEL,POINT (147492.700 171859.900),POINT (146776.500 171444.300),828.049395,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
3,001m,1,4,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,Comte de Flandre,Graaf van Vlaanderen,148013.6,171590.4,M,1,STOCKEL,POINT (148013.600 171590.400),POINT (147492.700 171859.900),586.487050,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
4,001m,1,5,8272,SAINTE-CATHERINE,SINT-KATELIJNE,Sainte-Catherine,Sint-Katelijne,148539.5,171278.2,M,1,STOCKEL,POINT (148539.500 171278.200),POINT (148013.600 171590.400),611.587811,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,218b,2,16,2209,NICOLAY,NICOLAY,Nicolay,Nicolay,148995.0,172059.0,B,218,GARE CENTRALE,POINT (148995.000 172059.000),POINT (149109.000 172374.000),334.994030,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."
4172,218b,2,17,2835,YSER,IJZER,Yser,IJzer,148905.0,171857.0,B,218,GARE CENTRALE,POINT (148905.000 171857.000),POINT (148995.000 172059.000),221.142488,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."
4173,218b,2,18,1901,SAINT-LOUIS,SAINT-LOUIS,Saint-Louis,Saint-Louis,149490.0,171535.0,B,218,GARE CENTRALE,POINT (149490.000 171535.000),POINT (148905.000 171857.000),667.764180,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."
4174,218b,2,19,2221,DE BERLAIMONT,DE BERLAIMONT,de Berlaimont,de Berlaimont,149398.0,171051.0,B,218,GARE CENTRALE,POINT (149398.000 171051.000),POINT (149490.000 171535.000),492.666216,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."


In [8]:
sample = headway_qos[(headway_qos.route_id == 6) & (headway_qos.service_id == 237534000) & (headway_qos.direction_id == 1)].sort_values(by=["time_seconds"]).groupby("trip_id")


In [9]:
sample.get_group((113339169237534000))

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity
1493157,113339169237534000,05:04:00,05:04:00,3481,1,0,0,NaN,BUYL,NaN,50.817420,4.378614,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18240,NaN,cluster_0,8.000000,1
1921976,113339169237534000,05:05:06,05:05:06,5208,2,0,0,NaN,CAMBRE-ETOILE,NaN,50.816372,4.376301,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18306,NaN,cluster_0,8.366667,1
128263,113339169237534000,05:07:00,05:07:00,1049F,3,0,0,NaN,LEGRAND,NaN,50.815519,4.371065,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18420,NaN,cluster_0,9.000000,1
1923783,113339169237534000,05:08:05,05:08:05,5210,4,0,0,NaN,BASCULE,NaN,50.814979,4.367815,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18485,NaN,cluster_0,9.000000,1
1924831,113339169237534000,05:10:00,05:10:00,5211,5,0,0,NaN,LONGCHAMP,NaN,50.811923,4.367234,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18600,NaN,cluster_0,9.000000,1
1925879,113339169237534000,05:11:15,05:11:15,5212,6,0,0,NaN,GOSSART,NaN,50.811392,4.361586,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18675,NaN,cluster_0,9.000000,1
1926927,113339169237534000,05:12:19,05:12:19,5213,7,0,0,NaN,CAVELL,NaN,50.811554,4.356734,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18739,NaN,cluster_0,9.000000,1
1929132,113339169237534000,05:12:47,05:12:47,5220F,8,0,0,NaN,CHURCHILL,NaN,50.811805,4.354662,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18767,NaN,cluster_0,9.000000,1
2817454,113339169237534000,05:13:00,05:13:00,9969F,9,0,0,NaN,CHURCHILL,NaN,50.811985,4.353655,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18780,NaN,cluster_0,9.000000,1
1928090,113339169237534000,05:13:10,05:13:10,5219F,10,0,0,NaN,CHURCHILL,NaN,50.812147,4.352676,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944461,007t0409,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,18790,NaN,cluster_0,9.000000,1


## Cleaning Stops ...

In [41]:
stops[stops.stop_name == "HEYSEL"]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1134,2740,NaN,HEYSEL,NaN,50.897503,4.334565,NaN,NaN,0,NaN
1135,2741,NaN,HEYSEL,NaN,50.897557,4.335408,NaN,NaN,0,NaN
1726,4264B,NaN,HEYSEL,NaN,50.896824,4.336515,NaN,NaN,0,NaN
1727,4265,NaN,HEYSEL,NaN,50.896429,4.335858,NaN,NaN,0,NaN
2825,8823,NaN,HEYSEL,NaN,50.897076,4.335408,NaN,NaN,0,59.0
2830,8824,NaN,HEYSEL,NaN,50.897198,4.335515,NaN,NaN,0,59.0
2839,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0,NaN
2840,9056F,NaN,HEYSEL,NaN,50.896415,4.338161,NaN,NaN,0,NaN
2962,59,NaN,HEYSEL,NaN,50.897076,4.335408,NaN,NaN,1,NaN


In [42]:
stops.stop_id.unique().shape

(2998,)

In [43]:
stops["stop_id_cleaned"] = stops.stop_id.apply(lambda x : re.sub('\D', '', x.lstrip("0")))
stop_cleaned = stops.drop_duplicates(subset = "stop_id_cleaned")
stop_cleaned.to_pickle("data/computed/gtfs3_/stops.pkl")

In [44]:
stop_cleaned[stop_cleaned.stop_name == "DIAMANT"]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned
69,0821,NaN,DIAMANT,NaN,50.848931,4.402428,NaN,NaN,0,68.0,821
72,0826,NaN,DIAMANT,NaN,50.849394,4.402198,NaN,NaN,0,68.0,826
2381,6448,NaN,DIAMANT,NaN,50.849465,4.402304,NaN,NaN,0,NaN,6448
2382,6449,NaN,DIAMANT,NaN,50.848712,4.401753,NaN,NaN,0,NaN,6449
2383,6450,NaN,DIAMANT,NaN,50.848802,4.401668,NaN,NaN,0,NaN,6450
2384,6451,NaN,DIAMANT,NaN,50.849117,4.401611,NaN,NaN,0,NaN,6451
2386,6454,NaN,DIAMANT,NaN,50.849234,4.401540,NaN,NaN,0,NaN,6454
2949,68,NaN,DIAMANT,NaN,50.848931,4.402428,NaN,NaN,1,NaN,68


In [10]:
stop_cleaned = pd.read_pickle("data/computed/gtfs3_/stops.pkl")
stop_cleaned

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned
0,0089,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,89
1,0470F,NaN,SIMONIS,NaN,50.863666,4.329612,NaN,NaN,0,NaN,470
2,0471,NaN,SIMONIS,NaN,50.863732,4.329236,NaN,NaN,0,53.0,471
3,0472,NaN,SIMONIS,NaN,50.863543,4.329023,NaN,NaN,0,53.0,472
4,0473F,NaN,SIMONIS,NaN,50.863418,4.330031,NaN,NaN,0,NaN,473
...,...,...,...,...,...,...,...,...,...,...,...
2993,34,NaN,TOMBERG,NaN,50.844316,4.425819,NaN,NaN,1,NaN,34
2994,44,NaN,TRONE,NaN,50.840813,4.366354,NaN,NaN,1,NaN,44
2995,32,NaN,VANDERVELDE,NaN,50.847387,4.446705,NaN,NaN,1,NaN,32
2996,6,NaN,VEEWEYDE,NaN,50.829391,4.300578,NaN,NaN,1,NaN,6


In [46]:
pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "left",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
)

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned
0,19-09-2021,10:33:18,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0.0,16.0,8022
1,19-09-2021,10:33:18,1,1,8732,8161,8732,NaN,GARE DE L'OUEST,NaN,50.848979,4.321096,NaN,NaN,0.0,2.0,8732
2,19-09-2021,10:33:18,1,1,8071,8731,8071,NaN,MERODE,NaN,50.839534,4.397696,NaN,NaN,0.0,21.0,8071
3,19-09-2021,10:33:18,1,0,8152,8161,8152,NaN,CRAINHEM,NaN,50.848478,4.458929,NaN,NaN,0.0,30.0,8152
4,19-09-2021,10:33:18,1,1,8271,8731,8271,NaN,SAINTE-CATHERINE,NaN,50.852347,4.347749,NaN,NaN,0.0,14.0,8271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421878,14-09-2021,12:25:54,97,232,6109,6361,6109,NaN,MORIS,NaN,50.824156,4.350302,NaN,NaN,0.0,NaN,6109
19421879,14-09-2021,12:25:54,98,0,3810,1951,3810,NaN,BOLLINCKX,NaN,50.813705,4.307014,NaN,NaN,0.0,NaN,3810
19421880,14-09-2021,12:25:54,98,0,1951,2382,1951,NaN,HEROS,NaN,50.803326,4.342649,NaN,NaN,0.0,NaN,1951
19421881,14-09-2021,12:25:54,98,270,3815,2382,3815,NaN,MOZART,NaN,50.810729,4.305499,NaN,NaN,0.0,NaN,3815


In [11]:
obs = pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "inner",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
)

In [48]:
obs

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned
0,19-09-2021,10:33:18,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
1,19-09-2021,10:38:41,5,0,8022,8262,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
2,19-09-2021,10:39:15,5,0,8022,8262,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
3,19-09-2021,10:40:51,1,0,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
4,19-09-2021,10:41:22,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19138654,14-09-2021,23:54:28,89,367,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208
19138655,14-09-2021,23:55:00,89,503,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208
19138656,15-09-2021,21:33:25,89,64,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208
19138657,15-09-2021,21:33:56,89,350,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208


In [12]:
obs_7 = obs[obs.route_short_name == "7"]
obs_7

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned
199070,19-09-2021,19:55:49,7,0,9959,5354,9959F,NaN,CHURCHILL,NaN,50.811688,4.354123,NaN,NaN,0,NaN,9959
199071,19-09-2021,19:56:23,7,0,9959,5354,9959F,NaN,CHURCHILL,NaN,50.811688,4.354123,NaN,NaN,0,NaN,9959
199072,19-09-2021,19:56:56,7,0,9959,5354,9959F,NaN,CHURCHILL,NaN,50.811688,4.354123,NaN,NaN,0,NaN,9959
199073,19-09-2021,19:57:29,7,0,9959,5354,9959F,NaN,CHURCHILL,NaN,50.811688,4.354123,NaN,NaN,0,NaN,9959
199074,19-09-2021,19:58:03,7,0,9959,5354,9959F,NaN,CHURCHILL,NaN,50.811688,4.354123,NaN,NaN,0,NaN,9959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19049058,21-09-2021,04:56:37,7,367,6485,6421,6485G,NaN,ESPLANADE,NaN,50.899576,4.348519,NaN,NaN,0,NaN,6485
19049059,21-09-2021,04:57:08,7,502,6485,6421,6485G,NaN,ESPLANADE,NaN,50.899576,4.348519,NaN,NaN,0,NaN,6485
19049060,21-09-2021,04:57:40,7,610,6485,6421,6485G,NaN,ESPLANADE,NaN,50.899576,4.348519,NaN,NaN,0,NaN,6485
19049061,21-09-2021,04:58:10,7,713,6485,6421,6485G,NaN,ESPLANADE,NaN,50.899576,4.348519,NaN,NaN,0,NaN,6485


In [50]:
obs_7.stop_id_x.unique()

array(['9959', '9986', '5706', '5770', '5701', '5267', '5354', '5219',
       '5210', '5758', '9056', '5307', '801', '9963', '1048', '5704',
       '5205', '816', '5312', '3480', '5771', '5355', '6421', '5705',
       '826', '906', '5211', '5308', '5311', '5264', '5200', '5356',
       '5266', '2246', '5352', '5773', '5212', '821', '5261', '6424',
       '5357', '5221', '5772', '811', '5258', '5265', '5806', '5700',
       '5350', '9969', '3481', '5707', '5255', '806', '5359', '5208',
       '2636', '901', '6456', '1049', '2635', '6457', '2351', '2243',
       '5303', '9051', '5317', '5222', '5305', '5253', '5306', '5254',
       '5213', '5220', '5256', '5759', '6422', '6485'], dtype=object)

We could detect more stops in the observed because the table stops was adding a letter. We drop the duplicates stops when removing the letter from the ID. Anyway the stop was the same and just slightly shifting in location.

## Schedule Matching

### One vehicule

Theoretically

In [51]:
geom[(geom.Code_Ligne == "007t") & (geom.VARIANTE == 2)]

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape
318,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,T,7,VANDERKINDERE,POINT (147847.800 176230.400),None,NaN,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
319,007t,2,2,2636F,SAINT-LAMBERT,SINT-LAMBERTUS,Saint-Lambert,Sint-Lambertus,148403.0,175613.0,T,7,VANDERKINDERE,POINT (148403.000 175613.000),POINT (147847.800 176230.400),830.319095,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
320,007t,2,3,5700G,DE WAND,DE WAND,De Wand,De Wand,149205.7,176083.5,T,7,VANDERKINDERE,POINT (149205.700 176083.500),POINT (148403.000 175613.000),930.428686,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
321,007t,2,4,5704F,ARAUCARIA,ARAUCARIA,Araucaria,Araucaria,149662.5,175835.7,T,7,VANDERKINDERE,POINT (149662.500 175835.700),POINT (149205.700 176083.500),519.683635,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
322,007t,2,5,5705,BUISSONNETS,BRAAMBOSJES,Buissonnets,Braambosjes,150008.0,175660.0,T,7,VANDERKINDERE,POINT (150008.000 175660.000),POINT (149662.500 175835.700),387.609004,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
323,007t,2,6,5706,HEEMBEEK,HEEMBEEK,Heembeek,Heembeek,150457.0,175346.0,T,7,VANDERKINDERE,POINT (150457.000 175346.000),POINT (150008.000 175660.000),547.902364,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
324,007t,2,7,5707,VAN PRAET,VAN PRAET,Van Praet,Van Praet,150320.0,174580.0,T,7,VANDERKINDERE,POINT (150320.000 174580.000),POINT (150457.000 175346.000),778.154869,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
325,007t,2,8,5758F,DOCKS BRUXSEL,DOCKS BRUXSEL,Docks Bruxsel,Docks Bruxsel,150428.0,174312.0,T,7,VANDERKINDERE,POINT (150428.000 174312.000),POINT (150320.000 174580.000),288.942901,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
326,007t,2,9,6457F,PRINC. ELISABETH,PRINS. ELISABETH,Princesse Élisabeth,Prinses Elisabeth,150659.0,173625.0,T,7,VANDERKINDERE,POINT (150659.000 173625.000),POINT (150428.000 174312.000),724.796523,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
327,007t,2,10,5303,DEMOLDER,DEMOLDER,Demolder,Demolder,150830.0,173492.0,T,7,VANDERKINDERE,POINT (150830.000 173492.000),POINT (150659.000 173625.000),216.633331,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."


In [13]:
sample = ht[(ht.route_id == 6) & (ht.service_id == 237534000) & (ht.direction_id == 1)].sort_values(by=["time_seconds"]).groupby("trip_id")
s = sample.get_group((113339244237534000))
s

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label
283240,113339244237534000,05:22:00,05:22:00,9056F,1,0,1,NaN,HEYSEL,NaN,50.896415,4.338161,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19320,NaN,cluster_0,9.000000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
274624,113339244237534000,05:24:00,05:24:00,2636F,2,0,0,NaN,SAINT-LAMBERT,NaN,50.890867,4.346055,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19440,NaN,cluster_0,9.000000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
280597,113339244237534000,05:27:00,05:27:00,5700G,3,0,0,NaN,DE WAND,NaN,50.895098,4.357463,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19620,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
280754,113339244237534000,05:28:29,05:28:29,5704F,4,0,0,NaN,ARAUCARIA,NaN,50.892871,4.363956,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19709,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
280911,113339244237534000,05:29:31,05:29:31,5705,5,0,0,NaN,BUISSONNETS,NaN,50.891291,4.368867,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19771,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
281068,113339244237534000,05:31:00,05:31:00,5706,6,0,0,NaN,HEEMBEEK,NaN,50.888468,4.375248,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19860,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
281225,113339244237534000,05:34:00,05:34:00,5707,7,0,0,NaN,VAN PRAET,NaN,50.881583,4.373300,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20040,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
281382,113339244237534000,05:35:04,05:35:04,5758F,8,0,0,NaN,DOCKS BRUXSEL,NaN,50.879173,4.374834,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20104,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
282928,113339244237534000,05:38:00,05:38:00,6457F,9,0,0,NaN,PRINC. ELISABETH,NaN,50.872997,4.378115,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20280,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
278242,113339244237534000,05:38:34,05:38:34,5303,10,0,0,NaN,DEMOLDER,NaN,50.871801,4.380544,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20314,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays


In [53]:
pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "left",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
).merge(
    stop_cleaned,
    "left",
    left_on = "stop_id_terminus",
    right_on = "stop_id_cleaned"
)

MemoryError: 

In [ ]:
pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "inner",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
).merge(
    stop_cleaned,
    "left",
    left_on = "stop_id_terminus",
    right_on = "stop_id_cleaned"
)

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code_x,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_cleaned_x,stop_id,stop_code_y,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y,stop_id_cleaned_y
0,19-09-2021,10:33:18,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161
1,19-09-2021,10:38:41,5,0,8022,8262,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8262,NaN,HERRMANN-DEBROUX,NaN,50.812243,4.429236,NaN,NaN,0.0,28.0,8262
2,19-09-2021,10:39:15,5,0,8022,8262,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8262,NaN,HERRMANN-DEBROUX,NaN,50.812243,4.429236,NaN,NaN,0.0,28.0,8262
3,19-09-2021,10:40:51,1,0,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161
4,19-09-2021,10:41:22,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19138654,14-09-2021,23:54:28,89,367,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208,2585,NaN,JACQUES BREL,NaN,50.842708,4.317885,NaN,NaN,0.0,NaN,2585
19138655,14-09-2021,23:55:00,89,503,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208,2585,NaN,JACQUES BREL,NaN,50.842708,4.317885,NaN,NaN,0.0,NaN,2585
19138656,15-09-2021,21:33:25,89,64,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208,2585,NaN,JACQUES BREL,NaN,50.842708,4.317885,NaN,NaN,0.0,NaN,2585
19138657,15-09-2021,21:33:56,89,350,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208,2585,NaN,JACQUES BREL,NaN,50.842708,4.317885,NaN,NaN,0.0,NaN,2585


In [ ]:
pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "inner",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
).merge(
    stop_cleaned,
    "inner",
    left_on = "stop_id_terminus",
    right_on = "stop_id_cleaned"
)

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code_x,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_cleaned_x,stop_id,stop_code_y,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y,stop_id_cleaned_y
0,19-09-2021,10:33:18,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0,29.0,8161
1,19-09-2021,10:40:51,1,0,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0,29.0,8161
2,19-09-2021,10:41:22,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0,29.0,8161
3,19-09-2021,10:47:56,1,0,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0,29.0,8161
4,19-09-2021,10:48:28,1,0,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0,29.0,8161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17704944,13-09-2021,13:30:34,57,0,2308,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308
17704945,13-09-2021,13:31:04,57,0,2308,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308
17704946,13-09-2021,13:31:36,57,0,2308,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308
17704947,13-09-2021,13:32:07,57,0,2308,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308,2308,NaN,VAN PRAET,NaN,50.881205,4.373115,NaN,NaN,0,NaN,2308


In [ ]:
pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "inner",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
)

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned
0,19-09-2021,10:33:18,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
1,19-09-2021,10:38:41,5,0,8022,8262,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
2,19-09-2021,10:39:15,5,0,8022,8262,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
3,19-09-2021,10:40:51,1,0,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
4,19-09-2021,10:41:22,1,1,8022,8161,8022,NaN,GARE CENTRALE,NaN,50.846349,4.358776,NaN,NaN,0,16.0,8022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18486174,14-09-2021,23:54:28,89,367,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208
18486175,14-09-2021,23:55:00,89,503,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208
18486176,15-09-2021,21:33:25,89,64,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208
18486177,15-09-2021,21:33:56,89,350,1208,2585,1208,NaN,HENRI REY,NaN,50.843505,4.305655,NaN,NaN,0,NaN,1208


In [14]:
stop_cleaned = pd.read_pickle("data/computed/gtfs3_/stops.pkl")

In [15]:
obs = pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "left",
    left_on = "stop_id",
    right_on = "stop_id_cleaned"
).merge(
    stop_cleaned,
    "left",
    left_on = "stop_id_terminus",
    right_on = "stop_id_cleaned"
)
obs_7 = obs[obs.route_short_name == "7"]
obs_7

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,stop_id_y,stop_code_x,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_cleaned_x,stop_id,stop_code_y,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y,stop_id_cleaned_y
52,19-09-2021,10:33:18,7,0,5267,9051,5267,NaN,ARSENAL,NaN,50.826578,4.397745,NaN,NaN,0.0,NaN,5267,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0.0,NaN,9051
53,19-09-2021,10:33:18,7,141,5354,9051,5354H,NaN,LEOPOLD III,NaN,50.857047,4.397456,NaN,NaN,0.0,NaN,5354,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0.0,NaN,9051
54,19-09-2021,10:33:18,7,34,5219,6421,5219F,NaN,CHURCHILL,NaN,50.812147,4.352676,NaN,NaN,0.0,NaN,5219,6421F,NaN,VANDERKINDERE,NaN,50.813378,4.348149,NaN,NaN,0.0,NaN,6421
55,19-09-2021,10:33:18,7,0,5210,9969,5210,NaN,BASCULE,NaN,50.814979,4.367815,NaN,NaN,0.0,NaN,5210,9969F,NaN,CHURCHILL,NaN,50.811985,4.353655,NaN,NaN,0.0,NaN,9969
56,19-09-2021,10:33:18,7,112,5770,9051,5770F,NaN,VAN PRAET,NaN,50.881268,4.373598,NaN,NaN,0.0,NaN,5770,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0.0,NaN,9051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421234,14-09-2021,12:25:54,7,528,5758,6421,5758F,NaN,DOCKS BRUXSEL,NaN,50.879173,4.374834,NaN,NaN,0.0,NaN,5758,6421F,NaN,VANDERKINDERE,NaN,50.813378,4.348149,NaN,NaN,0.0,NaN,6421
19421235,14-09-2021,12:25:54,7,73,5354,9051,5354H,NaN,LEOPOLD III,NaN,50.857047,4.397456,NaN,NaN,0.0,NaN,5354,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0.0,NaN,9051
19421236,14-09-2021,12:25:54,7,0,5806,9051,5806,NaN,DE WAND,NaN,50.895399,4.356785,NaN,NaN,0.0,NaN,5806,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0.0,NaN,9051
19421237,14-09-2021,12:25:54,7,564,806,9051,0806,NaN,MONTGOMERY,NaN,50.837555,4.407424,NaN,NaN,0.0,37.0,806,9051F,NaN,HEYSEL,NaN,50.896804,4.337345,NaN,NaN,0.0,NaN,9051


In [16]:
obs_7["date_label"] = np.where(
        pd.to_datetime(obs_7.date,format='%d-%m-%Y').dt.strftime('%A') == "Sunday",
        "sunday",
        np.where(
            pd.to_datetime(obs_7.date,format='%d-%m-%Y').dt.strftime('%A') == "Saturday",
            "saturday",
            "workdays"
        )
    )


In [17]:
time_sec_name = "time_seconds"
time = obs_7["time"].str.split(":", expand=True)
obs_7[time_sec_name] = (
    time[0].astype(int) * 3600 + time[1].astype(int) * 60 + time[2].astype(int)
)

In [ ]:
s.shape

(36, 46)

In [ ]:
s

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label
283240,113339244237534000,05:22:00,05:22:00,9056F,1,0,1,NaN,HEYSEL,NaN,50.896415,4.338161,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19320,NaN,cluster_0,9.000000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
274624,113339244237534000,05:24:00,05:24:00,2636F,2,0,0,NaN,SAINT-LAMBERT,NaN,50.890867,4.346055,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19440,NaN,cluster_0,9.000000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
280597,113339244237534000,05:27:00,05:27:00,5700G,3,0,0,NaN,DE WAND,NaN,50.895098,4.357463,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19620,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
280754,113339244237534000,05:28:29,05:28:29,5704F,4,0,0,NaN,ARAUCARIA,NaN,50.892871,4.363956,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19709,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
280911,113339244237534000,05:29:31,05:29:31,5705,5,0,0,NaN,BUISSONNETS,NaN,50.891291,4.368867,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19771,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
281068,113339244237534000,05:31:00,05:31:00,5706,6,0,0,NaN,HEEMBEEK,NaN,50.888468,4.375248,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,19860,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
281225,113339244237534000,05:34:00,05:34:00,5707,7,0,0,NaN,VAN PRAET,NaN,50.881583,4.373300,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20040,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
281382,113339244237534000,05:35:04,05:35:04,5758F,8,0,0,NaN,DOCKS BRUXSEL,NaN,50.879173,4.374834,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20104,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
282928,113339244237534000,05:38:00,05:38:00,6457F,9,0,0,NaN,PRINC. ELISABETH,NaN,50.872997,4.378115,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20280,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
278242,113339244237534000,05:38:34,05:38:34,5303,10,0,0,NaN,DEMOLDER,NaN,50.871801,4.380544,NaN,NaN,0,NaN,6,237534000,VANDERKINDERE,1,8944456,007t0634,7,VANDERKINDERE - HEYSEL,NaN,0,NaN,ECE354,000000,20314,29.0,cluster_0,8.500000,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays


In [18]:
def turn_sec_to_hours(time_in_sec: float) -> str:
    hours = int(time_in_sec / 3600)
    minutes = int(((time_in_sec / 3600) - hours) * 60)
    seconds = int(time_in_sec - hours * 3600 - minutes * 60)
    if len(str(minutes)) == 1:
        minutes = "0" + str(minutes)
    if len(str(hours)) == 1:
        hours = "0" + str(hours)
    if len(str(seconds)) == 1:
        seconds = "0" + str(seconds)
    return str(hours) + ":" + str(minutes) + ":" + str(seconds)

def extract_schedule(
    schedule: pd.DataFrame,
    transportation_type: int,
    thresh_gaps: int = 120,
    distance_col: str = "distance_from_point",
    speeds_ms: dict[int, float] = {
        3: 4.333333333333333,  # Bus
        0: 4.527777777777778,  # Tram
        1: 7.749999999999999,  # Metro
    },
) -> pd.DataFrame:
    if schedule.shape[0] > 1:

        schedule.loc[:,"lag_-1"] = schedule[distance_col].shift(1)
        schedule.loc[:,"lag_+1"] = schedule[distance_col].shift(-1)
        schedule.loc[:,"t_lag_before"] = schedule.sort_values("time_seconds")["time_seconds"].shift(1)
        schedule.loc[:,"t_diff(sec)"] = schedule["time_seconds"] - schedule["t_lag_before"]

        arr = schedule[(schedule["lag_-1"] > schedule[distance_col])]

        if (schedule.iloc[0][distance_col] > 0) | (
            schedule.iloc[0][distance_col] == 0 & schedule.iloc[1][distance_col] > 0
        ):
            # extracted_ = extracted_.append(schedule.iloc[0])
            arr = pd.concat([arr, schedule.head(1)])
        
        dep = schedule[(schedule["lag_+1"] > schedule[distance_col])]
        gaps = schedule[schedule["t_diff(sec)"] > thresh_gaps]

        FS = pd.concat([arr, dep, gaps])

    else:
        FS = schedule

    FS.loc[:,"adjusted_arrival_time(ts)"] = round(
        FS.time_seconds
        - (FS.distance_from_point / speeds_ms.get(transportation_type)),
        3,
    )
    FS.loc[:,"adjusted_arrival_time"] = FS["adjusted_arrival_time(ts)"].apply(
        turn_sec_to_hours
    )

    """
    if FS.iloc[0].stop_id_x == "5220":
        FS.to_csv("churchill_5220.csv")
    """

    return FS.sort_values(by="adjusted_arrival_time(ts)")

In [ ]:
g

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape
282,007t,1,1,6421F,VANDERKINDERE,VANDERKINDERE,Vanderkindere,Vanderkindere,148548.0,166993.0,T,7,HEYSEL,POINT (148548.000 166993.000),None,NaN,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
283,007t,1,2,5221F,CHURCHILL,CHURCHILL,Churchill,Churchill,148896.0,166844.0,T,7,HEYSEL,POINT (148896.000 166844.000),POINT (148548.000 166993.000),378.556469,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
284,007t,1,3,9963F,CHURCHILL,CHURCHILL,Churchill,Churchill,148976.0,166822.0,T,7,HEYSEL,POINT (148976.000 166822.000),POINT (148896.000 166844.000),82.969874,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
285,007t,1,4,5222F,CHURCHILL,CHURCHILL,Churchill,Churchill,149056.0,166804.0,T,7,HEYSEL,POINT (149056.000 166804.000),POINT (148976.000 166822.000),82.000000,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
286,007t,1,5,5253,CAVELL,CAVELL,Cavell,Cavell,149275.0,166773.0,T,7,HEYSEL,POINT (149275.000 166773.000),POINT (149056.000 166804.000),221.183182,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,007t,2,32,5213,CAVELL,CAVELL,Cavell,Cavell,149153.0,166790.0,T,7,VANDERKINDERE,POINT (149153.000 166790.000),POINT (149495.000 166772.000),342.473357,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
350,007t,2,33,5220F,CHURCHILL,CHURCHILL,Churchill,Churchill,149007.0,166818.0,T,7,VANDERKINDERE,POINT (149007.000 166818.000),POINT (149153.000 166790.000),148.660687,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
351,007t,2,34,9969F,CHURCHILL,CHURCHILL,Churchill,Churchill,148936.0,166838.0,T,7,VANDERKINDERE,POINT (148936.000 166838.000),POINT (149007.000 166818.000),73.763134,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
352,007t,2,35,5219F,CHURCHILL,CHURCHILL,Churchill,Churchill,148867.0,166856.0,T,7,VANDERKINDERE,POINT (148867.000 166856.000),POINT (148936.000 166838.000),71.309186,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."


In [ ]:
g

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape
282,007t,1,1,6421F,VANDERKINDERE,VANDERKINDERE,Vanderkindere,Vanderkindere,148548.0,166993.0,T,7,HEYSEL,POINT (148548.000 166993.000),None,NaN,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
283,007t,1,2,5221F,CHURCHILL,CHURCHILL,Churchill,Churchill,148896.0,166844.0,T,7,HEYSEL,POINT (148896.000 166844.000),POINT (148548.000 166993.000),378.556469,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
284,007t,1,3,9963F,CHURCHILL,CHURCHILL,Churchill,Churchill,148976.0,166822.0,T,7,HEYSEL,POINT (148976.000 166822.000),POINT (148896.000 166844.000),82.969874,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
285,007t,1,4,5222F,CHURCHILL,CHURCHILL,Churchill,Churchill,149056.0,166804.0,T,7,HEYSEL,POINT (149056.000 166804.000),POINT (148976.000 166822.000),82.000000,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
286,007t,1,5,5253,CAVELL,CAVELL,Cavell,Cavell,149275.0,166773.0,T,7,HEYSEL,POINT (149275.000 166773.000),POINT (149056.000 166804.000),221.183182,007t,1,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (148548.200 166992.700 0.000, 148..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,007t,2,32,5213,CAVELL,CAVELL,Cavell,Cavell,149153.0,166790.0,T,7,VANDERKINDERE,POINT (149153.000 166790.000),POINT (149495.000 166772.000),342.473357,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
350,007t,2,33,5220F,CHURCHILL,CHURCHILL,Churchill,Churchill,149007.0,166818.0,T,7,VANDERKINDERE,POINT (149007.000 166818.000),POINT (149153.000 166790.000),148.660687,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
351,007t,2,34,9969F,CHURCHILL,CHURCHILL,Churchill,Churchill,148936.0,166838.0,T,7,VANDERKINDERE,POINT (148936.000 166838.000),POINT (149007.000 166818.000),73.763134,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
352,007t,2,35,5219F,CHURCHILL,CHURCHILL,Churchill,Churchill,148867.0,166856.0,T,7,VANDERKINDERE,POINT (148867.000 166856.000),POINT (148936.000 166838.000),71.309186,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."


In [19]:
# det
g = geom[(geom.Code_Ligne == "007t")]
g_dir = pd.DataFrame()

for name, group in g.groupby("VARIANTE"):
    sched_stops = s.sort_values("time_seconds").stop_name.to_list()

    hset_fr = group.sort_values('succession').descr_fr.to_list()
    hset_nl = group.sort_values('succession').descr_nl.to_list()

    for i in range(len(hset_fr) - len(sched_stops) + 1):
        if (sched_stops == hset_fr[i:i+len(sched_stops)]) | (sched_stops == hset_nl[i:i+len(sched_stops)]):
            g_dir = group

g_dir["stop_id_cleaned"] = g_dir["stop_id"].apply(lambda x : re.sub('\D', '', x.lstrip("0")))


In [51]:
g_dir[g_dir.stop_id_cleaned == "9056"]

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape,stop_id_cleaned
318,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,T,7,VANDERKINDERE,POINT (147847.800 176230.400),None,NaN,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",9056


In [20]:
speeds_ms = {
    3: 4.333333333333333,  # Bus
    0: 4.527777777777778,  # Tram
    1: 7.749999999999999,  # Metro
}

d = []
times_ser = [0]
for row in s.to_dict('records'):
    stop_name = row['stop_name']
    stop_id = re.sub('\D', '', row["stop_id"].lstrip("0"))
    direction = row["trip_headsign"]
    cal_label = row["label"]
    transport_type = row["route_type"]
    expected_time = row["time_seconds"]
    stop_dist = g_dir[g_dir.stop_id_cleaned == stop_id].iloc[0].dist_m

    es = extract_schedule(obs_7[(obs_7.stop_name_x == stop_name) & (obs_7.date_label == cal_label) & (obs_7.date == "16-09-2021") & (obs_7.stop_id_x == stop_id)  & (obs_7.stop_name_y == direction)], transport_type)
    es.loc[:,"diff_time"] = abs(es["adjusted_arrival_time(ts)"] - expected_time)
    try:
        rec_matched = es[es.diff_time == min(es.diff_time)].to_dict('records')[0]
        t_diff = rec_matched["diff_time"]
        t_time = rec_matched["adjusted_arrival_time(ts)"]
        
        if (t_time < times_ser[-1]) & (t_diff < 10 * 60):
            print(stop_name, t_time, times_ser[-1], rec_matched["adjusted_arrival_time"])
            delta_time = stop_dist/speeds_ms.get(transport_type)
            rec_matched["adjusted_arrival_time(ts)"] =  times_ser[-1] + delta_time
            rec_matched["adjusted_arrival_time"] = turn_sec_to_hours(rec_matched["adjusted_arrival_time(ts)"])
            rec_matched["approx_rec"] = 1
            print(rec_matched["adjusted_arrival_time"])

        if (t_diff > 10 * 60):
            rec_matched = {}
        else :
            times_ser.append(t_time)
        
        rec_matched["dist_previous_stop"] = stop_dist

    except ValueError:
        print(row)
        print(es)
    
    temp = {**row, **rec_matched} if rec_matched != {} else row
    d.append(temp)

pd.DataFrame(d).to_csv("fourth_automated_match.csv")

BUISSONNETS 19882.442 19889.166 05:31:22
05:32:54
CAMBRE-ETOILE 21525.773 21887.0 05:58:45
06:05:31


### All the vehicules within a service ID

In [5]:
stop_cleaned = pd.read_pickle("data/computed/gtfs3_/stops.pkl")
exceptions = pd.read_pickle("data/computed/gtfs3_/calendar_dates.pkl")
geom = gpd.GeoDataFrame(pd.read_pickle("data/computed/stops_geom.pkl"))

obs = pd.read_pickle("data/computed/observed_schedule.pkl").merge(
    stop_cleaned,
    "left",
    left_on = "stop_id",
    right_on = "stop_id_cleaned",
    suffixes = [None, "__current"]
).merge(
    stop_cleaned,
    "left",
    left_on = "stop_id_terminus",
    right_on = "stop_id_cleaned",
    suffixes = [None, "__terminus"]
) 
obs = obs.drop(columns = ["stop_id","stop_id_terminus"])

ht = pd.read_pickle("data/computed/theoretical_schedule_qos.pkl").merge(
    pd.read_pickle("data/computed/gtfs3_/calendar.pkl"),
    "inner",
    "service_id"
)
ht = ht[ht.route_short_name.isin(obs.route_short_name.unique())]

In [6]:
# Theoretical schedule to search
r_name = "7"
service_ = 237534000
vehicle_ = 113339244237534000
sched = ht[(ht.route_short_name == r_name) & (ht.service_id == service_) & (ht.trip_id == vehicle_)]

In [4]:
# Sample to match

sample = obs[obs.route_short_name == "7"]

sample["date_label"] = np.where(
        pd.to_datetime(sample.date,format='%d-%m-%Y').dt.strftime('%A') == "Sunday",
        "sunday",
        np.where(
            pd.to_datetime(sample.date,format='%d-%m-%Y').dt.strftime('%A') == "Saturday",
            "saturday",
            "workdays"
        )
    )

time_sec_name = "time_seconds"
time = sample["time"].str.split(":", expand=True)
sample[time_sec_name] = (
    time[0].astype(int) * 3600 + time[1].astype(int) * 60 + time[2].astype(int)
)

In [5]:
def turn_sec_to_hours(time_in_sec: float) -> str:
    hours = int(time_in_sec / 3600)
    minutes = int(((time_in_sec / 3600) - hours) * 60)
    seconds = int(time_in_sec - hours * 3600 - minutes * 60)
    if len(str(minutes)) == 1:
        minutes = "0" + str(minutes)
    if len(str(hours)) == 1:
        hours = "0" + str(hours)
    if len(str(seconds)) == 1:
        seconds = "0" + str(seconds)
    return str(hours) + ":" + str(minutes) + ":" + str(seconds)

def extract_schedule(
    schedule: pd.DataFrame,
    transportation_type: int,
    thresh_gaps: int = 120,
    distance_col: str = "distance_from_point",
    speeds_ms: dict[int, float] = {
        3: 4.333333333333333,  # Bus
        0: 4.527777777777778,  # Tram
        1: 7.749999999999999,  # Metro
    },
) -> pd.DataFrame:
    if schedule.shape[0] > 1:

        schedule["lag_-1"] = schedule[distance_col].shift(1)
        schedule["lag_+1"] = schedule[distance_col].shift(-1)
        schedule["t_lag_before"] = schedule.sort_values("time_seconds")["time_seconds"].shift(1)
        schedule["t_diff(sec)"] = schedule["time_seconds"] - schedule["t_lag_before"]

        arr = schedule[(schedule["lag_-1"] > schedule[distance_col])]

        if (schedule.iloc[0][distance_col] > 0) | (
            schedule.iloc[0][distance_col] == 0 & schedule.iloc[1][distance_col] > 0
        ):
            arr = pd.concat([arr, schedule.head(1)])
        
        dep = schedule[(schedule["lag_+1"] > schedule[distance_col])]
        gaps = schedule[schedule["t_diff(sec)"] > thresh_gaps]

        FS = pd.concat([arr, dep, gaps])



    else:
        FS = schedule

    FS["adjusted_arrival_time(ts)"] = round(
        FS.time_seconds
        - (FS.distance_from_point / speeds_ms.get(transportation_type)),
        3,
    )
    FS["adjusted_arrival_time"] = FS["adjusted_arrival_time(ts)"].apply(
        turn_sec_to_hours
    )

    return FS.sort_values(by="adjusted_arrival_time(ts)")

def define_date_window(schedule_start_date:pd._libs.tslibs.timestamps.Timestamp ,schedule_end_date:pd._libs.tslibs.timestamps.Timestamp, label:str, service_id:int, exceptions:pd.DataFrame,  obs_start_date:str='2021-09-06 00:00:00', obs_end_date:str ='2021-09-21 00:00:00') -> list[str]:
    labels_dict = {
        "workdays" : ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
        "saturday" : ["Saturday"],
        "sunday" : ["Sunday"]
    }

    min_date = pd.to_datetime(obs_start_date)
    max_date = pd.to_datetime(obs_end_date)

    added_dates = pd.to_datetime(exceptions[(exceptions.service_id == service_id) & (exceptions.exception_type == 1)].date_ft)
    removed_dates = pd.to_datetime(exceptions[(exceptions.service_id == service_id) & (exceptions.exception_type == 2)].date_ft)

    service_window = pd.date_range(start= schedule_start_date, end= schedule_end_date)
    data_window = pd.date_range(start= min_date, end= max_date)

    woi_raw = [date for date in data_window if (date in service_window) & (date.strftime("%A") in labels_dict[label])]

    woi_raw.extend([dt for dt in added_dates if (dt not in woi_raw) & (dt not in data_window)])
    
    woi = [dt.strftime("%d-%m-%Y") for dt in woi_raw if dt not in removed_dates]

    return woi

def search_time(schedule:pd.DataFrame, expected_time:float, headway:float, stop_dist:float, speed_vehicle:float, time_last_val:float) -> dict:
    schedule["diff_time"] = abs(schedule["adjusted_arrival_time(ts)"] - expected_time)
    rec_matched = schedule[schedule.diff_time == min(schedule.diff_time)].to_dict('records')[0]

    t_diff = rec_matched["diff_time"]
    t_time = rec_matched["adjusted_arrival_time(ts)"]

    if (t_time < time_last_val) & (t_diff < headway * 60):
        delta_time = stop_dist/speed_vehicle
        rec_matched["adjusted_arrival_time(ts)"] =  time_last_val + delta_time
        rec_matched["adjusted_arrival_time"] = turn_sec_to_hours(rec_matched["adjusted_arrival_time(ts)"])
        rec_matched["approx_rec"] = 1

    return rec_matched


In [8]:
geom

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape
0,001m,1,1,8733,GARE DE L'OUEST,WESTSTATION,Gare de l'Ouest,Weststation,146633.5,170956.4,M,1,STOCKEL,POINT (146633.500 170956.400),None,NaN,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,001m,1,2,8742,BEEKKANT,BEEKKANT,Beekkant,Beekkant,146776.5,171444.3,M,1,STOCKEL,POINT (146776.500 171444.300),POINT (146633.500 170956.400),508.424439,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
2,001m,1,3,8292,ETANGS NOIRS,ZWARTE VIJVERS,Étangs Noirs,Zwarte Vijvers,147492.7,171859.9,M,1,STOCKEL,POINT (147492.700 171859.900),POINT (146776.500 171444.300),828.049395,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
3,001m,1,4,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,Comte de Flandre,Graaf van Vlaanderen,148013.6,171590.4,M,1,STOCKEL,POINT (148013.600 171590.400),POINT (147492.700 171859.900),586.487050,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
4,001m,1,5,8272,SAINTE-CATHERINE,SINT-KATELIJNE,Sainte-Catherine,Sint-Katelijne,148539.5,171278.2,M,1,STOCKEL,POINT (148539.500 171278.200),POINT (148013.600 171590.400),611.587811,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,218b,2,16,2209,NICOLAY,NICOLAY,Nicolay,Nicolay,148995.0,172059.0,B,218,GARE CENTRALE,POINT (148995.000 172059.000),POINT (149109.000 172374.000),334.994030,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."
4172,218b,2,17,2835,YSER,IJZER,Yser,IJzer,148905.0,171857.0,B,218,GARE CENTRALE,POINT (148905.000 171857.000),POINT (148995.000 172059.000),221.142488,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."
4173,218b,2,18,1901,SAINT-LOUIS,SAINT-LOUIS,Saint-Louis,Saint-Louis,149490.0,171535.0,B,218,GARE CENTRALE,POINT (149490.000 171535.000),POINT (148905.000 171857.000),667.764180,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."
4174,218b,2,19,2221,DE BERLAIMONT,DE BERLAIMONT,de Berlaimont,de Berlaimont,149398.0,171051.0,B,218,GARE CENTRALE,POINT (149398.000 171051.000),POINT (149490.000 171535.000),492.666216,218b,2,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (147730.000 176277.100 0.000, 147..."


In [24]:
set(obs.route_short_name.unique()) - set(ht.route_short_name.unique())

set()

In [26]:
set(obs.route_short_name.unique()) - set(geom.LIGNE.apply(lambda x : re.sub('\D', '', x.lstrip("0"))).unique())

set()

In [25]:
ht.route_short_name.unique()

array(['44', '39', '81', '9', '19', '82', '51', '3', '7', '97', '92',
       '62', '4', '25', '55', '13', '83', '52', '74', '89', '50', '49',
       '54', '14', '20', '87', '57', '88', '75', '98', '58', '59', '42',
       '45', '79', '80', '8', '93', '33', '48', '95', 'N04', 'N06', 'N05',
       'N09', 'N08', 'N10', 'N12', 'N11', 'N13', 'N16', '86', '27', '47',
       '38', '29', '46', '12', '21', '69', '64', '61', '28', '36', '53',
       '34', '43', '37', '70', '60', '56', '63', '65', '77', '76', '17',
       '66', '71', '72', '78', '41', 'T92', 'T7', 'N18', 'NAV', '5', '1',
       '6', '2'], dtype=object)

In [13]:
obs.route_short_name.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '12', '13', '14',
       '17', '19', '20', '21', '25', '27', '28', '29', '33', '34', '36',
       '37', '38', '39', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56', '58', '59', '60', '61',
       '62', '63', '64', '65', '66', '70', '71', '72', '74', '75', '76',
       '78', '79', '80', '81', '82', '83', '86', '87', '88', '89', '92',
       '93', '95', '97', '98', '57', '69', '77'], dtype=object)

In [11]:
geom.LIGNE.apply(lambda x : re.sub('\D', '', x.lstrip("0"))).unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '12', '13', '14',
       '17', '19', '20', '21', '25', '27', '28', '29', '32', '33', '34',
       '36', '37', '38', '39', '41', '42', '43', '44', '45', '46', '47',
       '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '60', '61', '62', '63', '64', '65', '66', '69', '70', '71',
       '72', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83',
       '86', '87', '88', '89', '92', '93', '95', '97', '98', '204', '205',
       '206', '208', '209', '210', '211', '212', '213', '216', '218'],
      dtype=object)

In [6]:
g = geom[(geom.Code_Ligne == "007t")]
g_dir = pd.DataFrame()

for name, group in g.groupby("VARIANTE"):
    sched_stops = sched.sort_values("time_seconds").stop_name.to_list()

    hset_fr = group.sort_values('succession').descr_fr.to_list()
    hset_nl = group.sort_values('succession').descr_nl.to_list()

    for i in range(len(hset_fr) - len(sched_stops) + 1):
        if (sched_stops == hset_fr[i:i+len(sched_stops)]) | (sched_stops == hset_nl[i:i+len(sched_stops)]):
            g_dir = group

g_dir["stop_id_cleaned"] = g_dir["stop_id"].apply(lambda x : re.sub('\D', '', x.lstrip("0")))

In [7]:
speeds_ms = {
    3: 4.333333333333333,  # Bus
    0: 4.527777777777778,  # Tram
    1: 7.749999999999999,  # Metro
}

d = []

sch = sched.iloc[0]
start_date = sch.start_date_ft
end_date = sch.end_date_ft
lab = sch.label
service_id = sch.service_id
doi = define_date_window(start_date, end_date, lab, service_id, exceptions)

for date in tqdm(doi):
    times_ser = [0]
    for row in sched.sort_values("time_seconds").to_dict('records'):
        stop_name = row['stop_name']
        stop_id = re.sub('\D', '', row["stop_id"].lstrip("0"))
        direction = row["trip_headsign"]
        cal_label = row["label"]
        transport_type = row["route_type"]
        expected_time = row["time_seconds"]
        headway = row["cluster_agg_value"]
        stop_dist = g_dir[g_dir.stop_id_cleaned == stop_id].iloc[0].dist_m

        es = extract_schedule(sample[(sample.stop_name == stop_name) & (sample.stop_id_cleaned == stop_id) & (sample.date == date) & (sample.stop_name__terminus == direction)], transport_type)

        if not es.empty:
            rec_matched = search_time(es, expected_time = expected_time, headway= headway, stop_dist = stop_dist, speed_vehicle = speeds_ms[transport_type], time_last_val=times_ser[-1])

            if (rec_matched["diff_time"] > headway * 60):
                rec_matched = {}
            else :
                times_ser.append(rec_matched["adjusted_arrival_time(ts)"])

            rec_matched["dist_previous_stop"] = stop_dist
            rec_matched["date_searched"] = date

            temp = {**row, **rec_matched} if rec_matched != {} else row
            d.append(temp)

pd.DataFrame(d).to_csv("data/CSVs/service_fullMatch_test_Date.csv")

100%|██████████| 10/10 [00:43<00:00,  4.35s/it]


In [ ]:
g = geom[(geom.Code_Ligne == "007t")]
g_dir = pd.DataFrame()

for name, group in g.groupby("VARIANTE"):
    sched_stops = sched.sort_values("time_seconds").stop_name.to_list()

    hset_fr = group.sort_values('succession').descr_fr.to_list()
    hset_nl = group.sort_values('succession').descr_nl.to_list()

    for i in range(len(hset_fr) - len(sched_stops) + 1):
        if (sched_stops == hset_fr[i:i+len(sched_stops)]) | (sched_stops == hset_nl[i:i+len(sched_stops)]):
            g_dir = group

g_dir["stop_id_cleaned"] = g_dir["stop_id"].apply(lambda x : re.sub('\D', '', x.lstrip("0")))

In [8]:
def detect_direction(ligne:str, geom:pd.DataFrame, schedule:pd.DataFrame) -> pd.DataFrame:
    g = geom[(geom.ligne_cleaned == ligne)]
    g_dir = pd.DataFrame()

    for name, group in g.groupby("VARIANTE"):
        sched_stops = schedule.sort_values("time_seconds").stop_name.to_list()

        hset_fr = group.sort_values('succession').descr_fr.to_list()
        hset_nl = group.sort_values('succession').descr_nl.to_list()

        for i in range(len(hset_fr) - len(sched_stops) + 1):
            if (sched_stops == hset_fr[i:i+len(sched_stops)]) | (sched_stops == hset_nl[i:i+len(sched_stops)]):
                g_dir = group

    return g_dir

In [9]:
detect_direction("7", geom, sched)

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,stop_Shape,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,ligne_Shape,stop_id_cleaned,ligne_cleaned
318,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,T,7,VANDERKINDERE,POINT (147847.800 176230.400),None,NaN,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",9056,7
319,007t,2,2,2636F,SAINT-LAMBERT,SINT-LAMBERTUS,Saint-Lambert,Sint-Lambertus,148403.0,175613.0,T,7,VANDERKINDERE,POINT (148403.000 175613.000),POINT (147847.800 176230.400),830.319095,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",2636,7
320,007t,2,3,5700G,DE WAND,DE WAND,De Wand,De Wand,149205.7,176083.5,T,7,VANDERKINDERE,POINT (149205.700 176083.500),POINT (148403.000 175613.000),930.428686,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5700,7
321,007t,2,4,5704F,ARAUCARIA,ARAUCARIA,Araucaria,Araucaria,149662.5,175835.7,T,7,VANDERKINDERE,POINT (149662.500 175835.700),POINT (149205.700 176083.500),519.683635,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5704,7
322,007t,2,5,5705,BUISSONNETS,BRAAMBOSJES,Buissonnets,Braambosjes,150008.0,175660.0,T,7,VANDERKINDERE,POINT (150008.000 175660.000),POINT (149662.500 175835.700),387.609004,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5705,7
323,007t,2,6,5706,HEEMBEEK,HEEMBEEK,Heembeek,Heembeek,150457.0,175346.0,T,7,VANDERKINDERE,POINT (150457.000 175346.000),POINT (150008.000 175660.000),547.902364,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5706,7
324,007t,2,7,5707,VAN PRAET,VAN PRAET,Van Praet,Van Praet,150320.0,174580.0,T,7,VANDERKINDERE,POINT (150320.000 174580.000),POINT (150457.000 175346.000),778.154869,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5707,7
325,007t,2,8,5758F,DOCKS BRUXSEL,DOCKS BRUXSEL,Docks Bruxsel,Docks Bruxsel,150428.0,174312.0,T,7,VANDERKINDERE,POINT (150428.000 174312.000),POINT (150320.000 174580.000),288.942901,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5758,7
326,007t,2,9,6457F,PRINC. ELISABETH,PRINS. ELISABETH,Princesse Élisabeth,Prinses Elisabeth,150659.0,173625.0,T,7,VANDERKINDERE,POINT (150659.000 173625.000),POINT (150428.000 174312.000),724.796523,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",6457,7
327,007t,2,10,5303,DEMOLDER,DEMOLDER,Demolder,Demolder,150830.0,173492.0,T,7,VANDERKINDERE,POINT (150830.000 173492.000),POINT (150659.000 173625.000),216.633331,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147...",5303,7
